In [ ]:
### Import the relevant Python libraries that we will use.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [ ]:
### Import the data set and evaluate the data structure.

In [ ]:
df = pd.read_csv("/churn_raw_data.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
### From the above, we can see a column that is unnamed, along with the case order column.
### Both of these columns can be dropped since they are not relevant to the churn rate. 
### This will reduce our columns from 52 to 50, keeping the 10,000 row count in tact. 
### The names of the survey response columns need to be addressed as well so that they cab be interpreted.
### Some statistics that we can see include: The average monthly charge is $174.08, The average customer 
### age is 53, The average income is $39,936.76 yearly, and All of the survey responses averaged around a 3.5.

In [ ]:
### We are missing key numerical data from the statistics, so let us look at the data types. 

In [ ]:
df.dtypes

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
### Data type inspection reveals a number of object typed variables that we will need to address.
### Next, we will see how many missing values there are currently.

In [ ]:
df.isnull().sum()

In [ ]:
### There are a number of missing values from the children, age, income, techie, phone, tech support, 
### tenure, and badnwidth GB year columns. Each column will be addressed individually. 
### Let's begin some of the clean-up. 

In [ ]:
### Dropping the unnamed column.
df = df.drop(['Unnamed: 0'], axis = 1)

### Dropping the case order column.
df = df.drop(['CaseOrder'], axis = 1)

df.shape

In [ ]:
### Now, let's change the column names for the survey responses using a dictionary.

In [ ]:
survey = {'item1': 'TimelyResponse', 
          'item2': 'TimelyFixes', 
          'item3': 'TimelyReplacements', 
          'item4': 'Reliability', 
          'item5': 'Options', 
          'item6': 'RespectfulResponse',
          'item7': 'CourteousExchange',
          'item8': 'EvidenceOfActiveListening'}
df.rename(columns = survey, inplace = True)
df.columns

In [ ]:
### Now, we can make sure that no rows are curently duplicates of each other.

In [ ]:
duplicates = df.loc[df.duplicated()]
print(duplicates)

In [ ]:
### Great news! There are no duplicated rows of data. Next, we need to address the missing values.
### Before we use any central tendencies to fill the missing values, we need to first detect outlying values.
### Each column (children, age, income, techie, phone, tech support,tenure, and bandwidth GB year) are 
### individually assessed.

In [ ]:
### Each numeric column (children, age, income, tenure, and bandwidth GB Year) is viewed in a box plot.
df.boxplot(['Children'])

In [ ]:
df.boxplot(['Age'])

In [ ]:
df.boxplot(['Income'])

In [ ]:
df.boxplot(['Tenure'])

In [ ]:
df.boxplot(['Bandwidth_GB_Year'])

In [ ]:
### When looking at this data, we are looking for extreme outlying values that will throw off the 
### central tendencies we use to fill in the missing values. From these box plots, we see nothing in age, 
### tenure or bandwidth GB year columns to worry about. The missing values in these columns can easily be 
### imputed with a central tendency like the median value. 
### We can see a few outliers in the children column, and several in the income column. With regards to our
### analysis, these variables will not effect the churn rate, have outliers, and have a number of missing
### values. Both the children and income columns should be dropped. 

In [ ]:
### Dropping the children column.
df = df.drop(['Children'], axis = 1)

### Dropping the income column.
df = df.drop(['Income'], axis = 1)

df.shape

In [ ]:
### Preserve a copy of the data frame to this point before imputing values.
df2 = df.copy()

In [ ]:
df2.head()

In [ ]:
### Imputing the median value into the missing values for the age column.
df2['Age'] = df2['Age'].fillna(df2['Age'].median())
df2['Age'].isnull().sum()

In [ ]:
### The missing values in the tenure and bandwidth columns could be due to the fact that the customner 
### has recently signed up for the service. The tenure does not begin to count until a month has passed.
### We can impute a zero in place of all of these missing values. 

In [ ]:
df2['Tenure'] = df2['Tenure'].fillna(0)
df2['Bandwidth_GB_Year'] = df2['Bandwidth_GB_Year'].fillna(0)
df2.isnull().sum()

In [ ]:
### Let's address the remaining three columns; techie, phone and tech support. For these we will find the
### value that most frequently occurs, and fill the missing values with that value.

In [ ]:
df2['Techie'].value_counts()

In [ ]:
df2['Techie'] = df2['Techie'].fillna('No')

In [ ]:
df2['Phone'].value_counts()

In [ ]:
df2['Phone'] = df2['Phone'].fillna('Yes')

In [ ]:
df2['TechSupport'].value_counts()

In [ ]:
df2['TechSupport'] = df2['TechSupport'].fillna('No')

In [ ]:
df2.isnull().sum()

In [ ]:
### We now have a complete and clean data set.

In [ ]:
### Extract the clean data before the PCA is done.
df2.to_csv('churn_clean.csv')

In [ ]:
### Creating a copy of the existing data frame with just the categorical variables.
df_cat = df2[['City', 'County', 'Zip', 'Job', 'Timezone', 'Lat', 'Lng', 'Customer_id', 'Interaction', 'State',
      'Gender', 'Churn', 'Techie', 'Contract', 'Port_modem', 'Area', 'Tablet', 'Phone', 'OnlineSecurity',
      'Multiple', 'OnlineBackup', 'TechSupport', 'DeviceProtection', 'StreamingTV', 'StreamingMovies', 
      'PaperlessBilling', 'PaymentMethod', 'Marital', 'InternetService', 'Employment', 'Education']].copy()

In [ ]:
### Reload the clean data, removing all categorical variables.
churn = pd.read_csv('churn_clean.csv')

In [ ]:
### Listing the categorical variables.
to_drop = ['City', 'County', 'Zip', 'Job', 'Timezone', 'Lat', 'Lng', 'Customer_id', 'Interaction', 'State',
      'Gender', 'Churn', 'Techie', 'Contract', 'Port_modem', 'Area', 'Tablet', 'Phone', 'OnlineSecurity',
      'Multiple', 'OnlineBackup', 'TechSupport', 'DeviceProtection', 'StreamingTV', 'StreamingMovies', 
      'PaperlessBilling', 'PaymentMethod', 'Marital', 'InternetService', 'Education', 'Employment']

### Drop the columns leaving only numerical data.
churn = churn.drop(columns=to_drop)
print(churn.head())

In [ ]:
### Because all of the columns have varying range values, we need to normalize the data. A pipeline will 
### do this task using the StandardScaler command in scikit learn.

In [ ]:
### Let's first take a look at the correlation of our data in a heatmap. 
sns.heatmap(churn.corr())

In [ ]:
### It looks like there is not much correlation currently. Bandwidth, tenure, timely responses and timely 
### fixes are the highest correlating factors from this graph. Options has the lowest correlating factor.

In [ ]:
### The PCA will consist of the following steps: StandardScaler and PCA will create a pipeline, The numerical
### data will be fit and transformed, The variance for each principal component will be calculated and plotted
### The number of components will be determined by the elbow method, The chosen number of components create 
### the PCA model and fit the data, The data is split into training and testing sets, Finally classifiers are
### applied to the pipeline and tested for prediction accuracy with and without the principal components.

In [ ]:
### Create pipeline and obtain principal components.
### Citation: (Scikit-Learn Documentation, 2022)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe1 = Pipeline([('scaler', StandardScaler()),('reducer',PCA())])
pc = pipe1.fit_transform(churn)
pc

In [ ]:
### Obtain the variances.
variances = pipe1.steps[1][1].explained_variance_ratio_*100
variances

In [ ]:
### Cumulative sum of variances.
np.cumsum(variances)

In [ ]:
### Scree plot of the variances.
plt.plot(variances)

In [ ]:
### From this scree plot, we can see that the first 3 and last 3 components are accounting for the data 
### variation. From tthis, we should keep 11 components for the analysis.

In [ ]:
### Split the training and test data.
from sklearn.model_selection import train_test_split
y = df_cat['Churn'].apply(lambda x:1 if x == 'Yes' else 0)
X_train, X_test, y_train, y_test = train_test_split(churn, y, test_size = 0.25)

In [ ]:
### Showing the variables of the training and test data.
print(X_train.head())
print(X_test.head())
print(y_train.head())
print(y_test.head())

In [ ]:
### Logistic regression and PCA to get the churn prediction accuracy.
from sklearn.linear_model import LogisticRegression

for i in range (3,15):
    pipe2 = Pipeline([('scaler',StandardScaler()), ('reducer',PCA(n_components=i)), 
                      ('classifier',LogisticRegression())])
    pipe2.fit(X_train,y_train)
    print(i, pipe2.score(X_test, y_test))

In [ ]:
### Logistic regression to get churn prediction without PCA.
pipe4 = Pipeline([('scaler', StandardScaler()), ('classifier', LogisticRegression())])
pipe4.fit(X_train, y_train)
pipe4.score(X_test, y_test)

In [ ]:
### Using logistic regression, we are obtaining 82.4% accuracy of prediction from our model.

In [ ]:
### Random forest classifier with PCA to calculate churn prediction accuracy.
from sklearn.ensemble import RandomForestClassifier

for i in range (3,15):
    pipe5 = Pipeline([('scaler',StandardScaler()),
                      ('reducer',PCA(n_components=i)),
                      ('classifier',RandomForestClassifier())])
    #pipe5.fit(X_train, y_train)
    X = X_train
    y = y_train    
    pipe5.fit(X,y)
    print(f"principal component: {i}, {pipe5.score(X_test, y_test)}")

In [ ]:
### Random forest classifier to calculate churn prediction accuracy without PCA.
pipe6 = Pipeline([('scaler',StandardScaler()), ('classifier',RandomForestClassifier())])
pipe6.fit(X_train, y_train)
print(pipe6.score(X_test, y_test))

In [ ]:
### From the random forest classifier, our model has an accuracy of 83.5%. 